In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
from itertools import product
import os


In [12]:
from utils import *
from helpers import *

In [4]:
from keras import backend as K
from keras.models import Sequential, Model  
from keras.layers import *
 
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model, load_model
from keras import optimizers
from keras.optimizers import Nadam
from keras.utils import Sequence

from keras_tqdm import TQDMNotebookCallback
from tqdm import tqdm_notebook


In [5]:
os.listdir('data')

['data.csv',
 'test_index_sample.csv',
 'train_index_sample.csv',
 'valid_index_sample.csv']

In [13]:
data_dir = 'data/'
data, data_train, data_valid, data_test = read_data(data_dir)

In [17]:
time_feat = ['day', 'hour',  'duration_std', 'time_betw_std']
cat_feat = ['case_applicationtype', 'case_loangoal', 'accepted', 'resource']
num_feat = ['case_requestedamount_std', 'creditscore_std','firstwithdrawalamount_std', 'monthlycost_std', 'numberofterms_std', 'offeredamount_std']

feat_dic = {}
feat_dic['time_feat'] = time_feat
feat_dic['cat_feat'] = cat_feat
feat_dic['num_feat'] = num_feat

print(time_feat)
print(cat_feat)
print(num_feat)

['day', 'hour', 'duration_std', 'time_betw_std']
['case_applicationtype', 'case_loangoal', 'accepted', 'resource']
['case_requestedamount_std', 'creditscore_std', 'firstwithdrawalamount_std', 'monthlycost_std', 'numberofterms_std', 'offeredamount_std']


In [18]:
helpers = get_helpers(data, feat_dic)

In [19]:
config_base_name = lambda config: ' '.join(["{}-{}".format(k,v) for k, v in config.items() if not k.startswith('_')])
config_file_template = lambda config: 'inter_latent50' + config_base_name(config) + ' {epoch:02d} {loss:.5f} {val_loss:.5f}.h5'

model_configurations = [ 
    {'output_dim': int(data_train.append(data_valid)['length'].max()), 'HUnits': units, 'dropout': do, 'lr': lr} for units, do, lr in product([40], [0], [0.002]) #output dim should be max sequence length
] 

In [20]:
def get_model(output_dim=25, dropout_rate=0.2, learning_rate = 0.002, latent_dim=40, helpers_dic=helpers):
    
    print('drop:', dropout_rate)
           
    inp_trace = Input(shape=(output_dim, helpers_dic['trace_helper']['vocab_size']), name='Xtrace')
    inp_day = Input(shape=(output_dim, 1), name='Xday')
    inp_hour = Input(shape=(output_dim, 1), name='Xhour')
    inp_duration = Input(shape=(output_dim, 1), name='Xduration_std')
    inp_time = Input(shape=(output_dim, 1), name='Xtime_betw_std')
    
    inp_application = Input(shape=(output_dim, 1), name='Xcase_applicationtype')
    inp_lgoal = Input(shape=(output_dim, 1), name='Xcase_loangoal')
    inp_reqamount = Input(shape=(output_dim, 1), name='Xcase_requestedamount_std')
    
    inp_accepted = Input(shape=(output_dim, 1), name='Xaccepted')
    inp_resource = Input(shape=(output_dim, 1), name='Xresource')
    inp_creditscore = Input(shape=(output_dim, 1), name='Xcreditscore_std')
    inp_firstwd = Input(shape=(output_dim, 1), name='Xfirstwithdrawalamount_std')
    inp_mcost = Input(shape=(output_dim, 1), name='Xmonthlycost_std')
    inp_nterms = Input(shape=(output_dim, 1), name='Xnumberofterms_std')
    inp_oamount = Input(shape=(output_dim, 1), name='Xofferedamount_std')
            
    emb_hour = Embedding(output_dim = round(1.6 * (helpers_dic['time_helpers']['hour']['size']-1)**0.56), input_dim=helpers_dic['time_helpers']['hour']['size'], name='EmbHour')(Flatten()(inp_hour))
    emb_day = Embedding(output_dim =  round(1.6 * (helpers_dic['time_helpers']['day']['size']-1)**0.56), input_dim=helpers_dic['time_helpers']['day']['size'], name='EmbDay')(Flatten()(inp_day))
    emb_application = Embedding(output_dim =  round(1.6 * (helpers_dic['cat_helpers']['case_applicationtype']['size']-1)**0.56), input_dim=helpers_dic['cat_helpers']['case_applicationtype']['size'], name='EmbApplication')(Flatten()(inp_application))
    emb_lgoal = Embedding(output_dim =  round(1.6 * (helpers_dic['cat_helpers']['case_loangoal']['size']-1)**0.56), input_dim=helpers_dic['cat_helpers']['case_loangoal']['size'], name='EmbLgoal')(Flatten()(inp_lgoal))
    emb_resource = Embedding(output_dim =  round(1.6 * (helpers_dic['cat_helpers']['resource']['size']-1)**0.56), input_dim=helpers_dic['cat_helpers']['resource']['size'], name='EmbResource')(Flatten()(inp_resource))
    emb_accepted = Embedding(output_dim =  round(1.6 * (helpers_dic['cat_helpers']['accepted']['size']-1)**0.56), input_dim=helpers_dic['cat_helpers']['accepted']['size'], name='EmbAccepted')(Flatten()(inp_accepted))
    
    
    merged = concatenate([(inp_trace), 
                          (emb_day), (emb_hour),(inp_duration), (inp_time),
                          (emb_application), (emb_lgoal), (inp_reqamount),
                          (emb_resource), (inp_creditscore), (emb_accepted), (inp_firstwd), (inp_mcost), (inp_nterms), (inp_oamount)], name='concat_input')
        
    
    LSTM1 = Bidirectional(LSTM(latent_dim, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, recurrent_dropout = dropout_rate, dropout = dropout_rate), name='LSTMshared1')
    outputs1 = LSTM1(merged)
    
    b1 = BatchNormalization()(outputs1)
    
    LSTM_trace1 = Bidirectional(LSTM(latent_dim,  implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, recurrent_dropout = dropout_rate, dropout = dropout_rate), name='LSTMtrace1') #maybe do return seq false
    outputs2_1a = LSTM_trace1(b1)
    
    b2_1a = BatchNormalization()(outputs2_1a)
    
    LSTM_time1 = Bidirectional(LSTM(latent_dim, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, recurrent_dropout = dropout_rate, dropout = dropout_rate), name='LSTMtime1') #maybe do return seq false
    outputs2_2a = LSTM_time1(b1)
    
    b2_2a = BatchNormalization()(outputs2_2a)
       
    output_trace = TimeDistributed(Dense(helpers_dic['trace_helper']['vocab_size'], kernel_initializer='glorot_uniform', activation='softmax'), name='trace_out')(b2_1a)
    
    output_time = TimeDistributed(Dense(1, kernel_initializer='he_uniform'), name='time_out_nolr')(b2_2a) #used to be name time_out_nolr with LeakyRelu
    output_time = TimeDistributed(PReLU(), name='time_out')(output_time)
        
    model = Model(inputs=[inp_trace, 
                          inp_day, inp_hour, inp_duration, inp_time,
                          inp_application, inp_lgoal, inp_reqamount, 
                          inp_resource, inp_creditscore, inp_accepted, inp_firstwd, inp_mcost, inp_nterms, inp_oamount], 
                  outputs=[output_time, output_trace])
    

    opt = Nadam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)#clipvalue=3
    
    
    model.compile(loss={'trace_out':'categorical_crossentropy', 'time_out':'mae'}, optimizer=opt) 
    return model

In [21]:
def train_model(config, epochs=500, feat_dic = feat_dic, helpers_dic=helpers):
    training_generator = BagDataGenerator(data_train,  config['output_dim'], feat_dic, helpers_dic)
    validation_generator = BagDataGenerator(data_valid,  config['output_dim'], feat_dic, helpers_dic)
    
    model_checkpoint = ModelCheckpoint(config_file_template(config), 
                                       monitor='val_loss', verbose=0, 
                                       save_best_only=True, save_weights_only=False, mode='auto')
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=16, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0) #was factor 0.5 and 0.01
    early_stopping = EarlyStopping(monitor='val_loss', patience=59)
    
   
    output_dim = config['output_dim']
    dropout = config['dropout']
    lr = config['lr']
    latent_dim = config['HUnits']
    
    
    print('*** Now trainining:\n', config_base_name(config))
    
    model = get_model(output_dim, dropout, lr, latent_dim)
    
    print(model.summary())
    
    training = model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        epochs=epochs, verbose=2,
                        class_weight=None,
                        use_multiprocessing = False,
                        callbacks=[model_checkpoint, early_stopping, lr_reducer]) 
    return training 





In [ ]:
for config in model_configurations:
    training = train_model(config) 

*** Now trainining:
 output_dim-63 HUnits-40 dropout-0 lr-0.002
drop: 0
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Xday (InputLayer)               (None, 63, 1)        0                                            
__________________________________________________________________________________________________
Xhour (InputLayer)              (None, 63, 1)        0                                            
__________________________________________________________________________________________________
Xcase_applicationtype (InputLay (None, 63, 1)        0                                            
__________________________________________________________________________________________________
Xcase_loangoal (InputLayer)     (None, 63, 1)        0          

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
